# Reinforcement Learning

### 1.Import dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
import os
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from stable_baselines3 import PPO  
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
Display().start()


In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

### 2. Load Environment

In [ ]:
environment_name = 'CartPole-v0'
env = wrap_env(gym.make(environment_name))
env.reset()


array([-0.04780231,  0.02397257,  0.03582421,  0.03651027])

In [ ]:
#loop to test our environment

episodes = 5
for episode in range(1, episodes + 1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{}, Score:{}'.format(episode, score))
env.close()
show_video()

Episode:1, Score:15.0
Episode:2, Score:92.0
Episode:3, Score:19.0
Episode:4, Score:36.0
Episode:5, Score:78.0


Understanding The Environment

In [ ]:
env.action_space

Discrete(2)

In [ ]:
env.observation_space

Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

### 3.Train RL model

In [ ]:
#make the directories first
log_path = os.path.join('gdrive','MyDrive','Training','Logs')

In [ ]:
log_path

'gdrive/MyDrive/Training/Logs'

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log= log_path)

Using cpu device


In [ ]:
model.learn(total_timesteps=20000)

Logging to gdrive/MyDrive/Training/Logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 1551 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1138        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007781787 |
|    clip_fraction        | 0.0833      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00155    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.34        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0117     |
|    value_loss           | 51.8        |
------------------------------

### 4. Save and Reload Model

In [ ]:
PPO_Path = os.path.join('gdrive','MyDrive','Training','Saved Models','PPO_Model_Cartpole')

In [ ]:
model.save(PPO_Path)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'gdrive/MyDrive/Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
del model

In [ ]:
model = PPO.load(PPO_Path, env= env)

### 5. Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

In [ ]:
env.close()

### 6.Test Model

In [ ]:
#loop to test our environment

episodes = 5
for episode in range(1, episodes + 1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    
    action, _ = model.predict(obs)   #now using model here
    obs, reward, done, info = env.step(action)
    score += reward
  print('Episode:{}, Score:{}'.format(episode, score))
env.close()
#show_video()

Episode:1, Score:[200.]
Episode:2, Score:[200.]
Episode:3, Score:[200.]
Episode:4, Score:[200.]
Episode:5, Score:[200.]


### 7. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path,'PPO_5')

In [ ]:
!pip install tensorboard

In [ ]:
!tensorboard --logdir={training_log_path}

### 8.Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


In [ ]:
save_path = os.path.join('gdrive','MyDrive','Training','Saved Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best= stop_callback, 
                             eval_freq=10000, best_model_save_path= save_path, verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log= log_path)
model.learn(total_timesteps=200000, callback= eval_callback)

Using cpu device
Logging to gdrive/MyDrive/Training/Logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 1516 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1118        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008894795 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000938    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.95        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 48.6        |
-------------

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=188.40 +/- 23.20
Episode length: 188.40 +/- 23.20
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 188         |
|    mean_reward          | 188         |
| time/                   |             |
|    total timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.005411352 |
|    clip_fraction        | 0.0411      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.618      |
|    explained_variance   | 0.32        |
|    learning_rate        | 0.0003      |
|    loss                 | 17.5        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.00976    |
|    value_loss           | 55.9        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 917   |
|    iterations      | 5     |
|    ti

### 9. Channging Policies

In [ ]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log= log_path, policy_kwargs={'net_arch':net_arch})
model.learn(total_timesteps=20000, callback= eval_callback)

Using cpu device
Logging to gdrive/MyDrive/Training/Logs/PPO_8
Eval num_timesteps=848, episode_reward=9.40 +/- 1.20
Episode length: 9.40 +/- 1.20
---------------------------------
| eval/              |          |
|    mean_ep_length  | 9.4      |
|    mean_reward     | 9.4      |
| time/              |          |
|    total timesteps | 848      |
---------------------------------


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


-----------------------------
| time/              |      |
|    fps             | 1298 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 830         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016075512 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.000477   |
|    learning_rate        | 0.0003      |
|    loss                 | 4.13        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0273     |
|    value_loss           | 20.4        |
-----------------------------------------
----------------------------------

### 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log= log_path)
model.learn(total_timesteps=20000)


Using cpu device
Logging to gdrive/MyDrive/Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration rate | 0.97     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2656     |
|    time_elapsed     | 0        |
|    total timesteps  | 64       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.924    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3849     |
|    time_elapsed     | 0        |
|    total timesteps  | 161      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.861    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4677     |
|    time_elapsed     | 0        |
|    total timesteps  | 292      |
---------------------------